In [1]:
input = open("inputs/5").read()

In [2]:
test_input = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [3]:
def parse_input(input):
    seed_str, *rest = input.split("\n\n")

    seeds = [int(s) for s in seed_str.split()[1:]]

    maps = []
    for map in rest:
        map_ranges = []
        lines = map.splitlines()[1:]

        for line in lines:
            map_ranges.append([int(n) for n in line.split()])

        maps.append(map_ranges)
    
    return seeds, maps

In [4]:
def p1(input):
    seeds, maps = parse_input(input)

    map_ranges = []

    for map in maps:
        ranges = []

        for (dest_start, source_start, range) in map:
            # ok so we want a mapping from source to destination number which is valid in given range of the source number
            # simplest format for my brain is to specify a range of [start, end) where the source gets transformed and then a constant representing the transformation
            # so, let's store as triple [start, end, constant]

            ranges.append([source_start, source_start + range, dest_start - source_start])
        
        map_ranges.append(ranges)


    # now let's just follow the map

    final_locations = []

    for seed in seeds:
        location = seed

        for map in map_ranges:
            for (start, end, constant) in map:
                if start <= location < end:
                    location += constant
                    break
            else:
                # location stays the same since it doesn't get modified
                pass

        final_locations.append(location)
    
    return min(final_locations)

In [5]:
p1(test_input)

35

In [6]:
p1(input)

1181555926

In [11]:
# OK I'm seeing the trick. The composition of piecewise linear functions is piecewise linear. So, we can just figure out the "critical points" of the resulting composition and then only check those points. The minimum will occur at one of those.

# To do that I think I have to compute critical points backwards
# initial critical points are the ends of the location range

In [58]:
def has_intersection(ranges):
    # integer ranges with half-open intervals. [a, b)

    n = len(ranges)
    for i in range(n):
        for j in range(i+1, n):
            # Unpack the ranges
            a, b = ranges[i]
            c, d = ranges[j]

            # Check for intersection
            if a < d and c < b:
                return True  # Intersection found
    return False  # No intersections found

In [63]:
def complete_intervals(intervals):
    # Sort the intervals by their start points
    intervals.sort(key=lambda x: x[0])

    # Initialize the completed list with an interval from 0 to the start of the first interval, if necessary
    completed = [(0, intervals[0][0])] if intervals[0][0] > 0 else []

    # Iterate through intervals to fill the gaps
    for i in range(len(intervals) - 1):
        completed.append(intervals[i])  # Add the current interval
        # Check if there is a gap to the next interval
        if intervals[i][1] < intervals[i + 1][0]:
            completed.append((intervals[i][1], intervals[i + 1][0]))

    # Add the last interval
    completed.append(intervals[-1])

    # Add an interval from the end of the last interval to infinity, if necessary
    if intervals[-1][1] != float('inf'):
        completed.append((intervals[-1][1], float('inf')))

    return completed

# Example usage
# intervals = [(1, 2), (2, 3), (6, 9)]
# intervals = [(1, 2), (2, 3), (6, 9), (9, float('inf'))]
intervals = [(0, 1), (1, 2), (2, 3), (6, 9), (9, float('inf'))]
completed_intervals = complete_intervals(intervals)
completed_intervals

[(0, 1), (1, 2), (2, 3), (3, 6), (6, 9), (9, inf)]

In [94]:
def complete_intervals_with_constant(intervals):
    # Sort the intervals by their start points
    intervals.sort(key=lambda x: x[0])

    # Initialize the completed list with an interval from 0 to the start of the first interval, if necessary
    completed = [(0, intervals[0][0], 0)] if intervals[0][0] > 0 else []

    # Iterate through intervals to fill the gaps
    for i in range(len(intervals) - 1):
        completed.append(intervals[i])  # Add the current interval
        # Check if there is a gap to the next interval
        if intervals[i][1] < intervals[i + 1][0]:
            completed.append((intervals[i][1], intervals[i + 1][0], 0))

    # Add the last interval
    completed.append(intervals[-1])

    # Add an interval from the end of the last interval to infinity, if necessary
    if intervals[-1][1] != float('inf'):
        completed.append((intervals[-1][1], float('inf'), 0))

    return completed

# Example usage
intervals = [(1, 2, 5), (2, 3, 10), (6, 9, 15)]
completed_intervals = complete_intervals_with_constant(intervals)
completed_intervals

[(0, 1, 0), (1, 2, 5), (2, 3, 10), (3, 6, 0), (6, 9, 15), (9, inf, 0)]

In [101]:

import math
from tqdm import tqdm
from itertools import batched

def p2(input):
    seeds, maps = parse_input(input)

    # pair into tuples of two
    seed_ranges = list(batched(seeds, 2))

    map_ranges = []

    for map in maps:
        forward_ranges = []
        backward_ranges = []

        for (dest_start, source_start, rng) in map:
            # ok so we want a mapping from source to destination number which is valid in given range of the source number
            # simplest format for my brain is to specify a range of [start, end) where the source gets transformed and then a constant representing the transformation
            # so, let's store as triple [start, end, constant]

            c = dest_start - source_start

            forward_ranges.append([source_start, source_start + rng, c])
            backward_ranges.append([dest_start, dest_start + rng, -c])

        map_ranges.append((complete_intervals_with_constant(forward_ranges), complete_intervals_with_constant(backward_ranges)))
    
    # sanity checking this
    for (completed_forward_ranges, completed_backward_ranges) in map_ranges[::-1]:

        print("Codomain ranges:")
        print(completed_backward_ranges)
        print("Domain ranges:")
        print(completed_forward_ranges)
        print()

        completed_backward_ranges_interval_only = [(a, b) for (a, b, c) in completed_backward_ranges]
        completed_forward_ranges_interval_only = [(a, b) for (a, b, c) in completed_forward_ranges]

        if has_intersection(completed_backward_ranges_interval_only):
            assert False, "Completed backwards has intersection"

        if has_intersection(completed_forward_ranges_interval_only):
            assert False, "Completed forwards has intersection"
    

In [102]:
# p2(input)
p2(test_input)

Codomain ranges:
[(0, 56, 0), [56, 60, 37], [60, 97, -4], (97, inf, 0)]
Domain ranges:
[(0, 56, 0), [56, 93, 4], [93, 97, -37], (97, inf, 0)]

Codomain ranges:
[[0, 1, 69], [1, 70, -1], (70, inf, 0)]
Domain ranges:
[[0, 69, 1], [69, 70, -69], (70, inf, 0)]

Codomain ranges:
[(0, 45, 0), [45, 68, 32], [68, 81, -4], [81, 100, -36], (100, inf, 0)]
Domain ranges:
[(0, 45, 0), [45, 64, 36], [64, 77, 4], [77, 100, -32], (100, inf, 0)]

Codomain ranges:
[(0, 18, 0), [18, 88, 7], [88, 95, -70], (95, inf, 0)]
Domain ranges:
[(0, 18, 0), [18, 25, 70], [25, 95, -7], (95, inf, 0)]

Codomain ranges:
[[0, 42, 11], [42, 49, -42], [49, 57, 4], [57, 61, -50], (61, inf, 0)]
Domain ranges:
[[0, 7, 42], [7, 11, 50], [11, 53, -11], [53, 61, -4], (61, inf, 0)]

Codomain ranges:
[[0, 37, 15], [37, 39, 15], [39, 54, -39], (54, inf, 0)]
Domain ranges:
[[0, 15, 39], [15, 52, -15], [52, 54, -15], (54, inf, 0)]

Codomain ranges:
[(0, 50, 0), [50, 52, 48], [52, 100, -2], (100, inf, 0)]
Domain ranges:
[(0, 50, 0), 

In [64]:
import math
from tqdm import tqdm
from itertools import batched

def p2(input):
    seeds, maps = parse_input(input)

    # pair into tuples of two
    seed_ranges = list(batched(seeds, 2))

    map_ranges = []

    for map in maps:
        forward_ranges = []
        backward_ranges = []

        for (dest_start, source_start, rng) in map:
            # ok so we want a mapping from source to destination number which is valid in given range of the source number
            # simplest format for my brain is to specify a range of [start, end) where the source gets transformed and then a constant representing the transformation
            # so, let's store as triple [start, end, constant]

            c = dest_start - source_start

            forward_ranges.append([source_start, source_start + rng, c])
            backward_ranges.append([dest_start, dest_start + rng, -c])

        print(backward_ranges)
        map_ranges.append((forward_ranges, backward_ranges))

    # now let's build critical points and go backwards with the backwards range

    critical_points = []

    last_forward_ranges = map_ranges[-1][0]
    for (start, end, constant) in last_forward_ranges:
        critical_points.append(start)
        critical_points.append(end-1)

    for (forward_ranges, backward_ranges) in map_ranges[::-1][1:]:
        new_critical_points = []

        for point in critical_points:
            for (start, end, constant) in backward_ranges:
                if start <= point < end:
                    new_critical_points.append(point + constant)

                    # backwards ranges can overlap, so we don't break now
                    # break
            
            # if the point doesn't show up in any forward ranges then we can also add itself back
            for (start, end, constant) in forward_ranges:
                if start <= point < end:
                    # print('found in range; its not in a y=x')
                    break
            else:
                # print('didnt find in range; it is in y=x')
                new_critical_points.append(point)
        
        print(len(new_critical_points))
        # print(sorted(new_critical_points))

        critical_points = new_critical_points
    

    def eval_start_seed(seed):
        location = seed

        for (forward_ranges, _backward_ranges) in map_ranges:
            for (start, end, constant) in forward_ranges:
                if start <= location < end:
                    location += constant
                    break
            else:
                # location stays the same since it doesn't get modified
                pass
        
        return location
    

In [56]:
p2(test_input)

[[50, 52, 48], [52, 100, -2]]
[[0, 37, 15], [37, 39, 15], [39, 54, -39]]
[[49, 57, 4], [0, 42, 11], [42, 49, -42], [57, 61, -50]]
[[88, 95, -70], [18, 88, 7]]
[[45, 68, 32], [81, 100, -36], [68, 81, -4]]
[[0, 1, 69], [1, 70, -1]]
[[60, 97, -4], [56, 60, 37]]
4
4
4
4
4
4
54 55 60 68 69
94 95 96 56 57
95 96 56 57 58
57 58 59 97 98


In [65]:
p2(input)

[[2069473506, 2070957389, 1663113949], [3235691256, 3242241597, -886701136], [3547561069, 4294967296, -2155365398], [3264251584, 3547561069, 469819754], [391285622, 586838162, -133528050], [1645243555, 2022435244, 1521714765], [335002083, 391285622, 177208786], [3242241597, 3264251584, -2344506508], [77244511, 335002083, -77244511], [989159646, 1112103608, 3182863688], [605476380, 793913826, 2938673629], [0, 18343754, 568494408], [2700122696, 2821869347, 1350153987], [2022435244, 2069473506, 117166654], [2227672101, 2323512370, -1307927025], [1112103608, 1645243555, 1521714765], [826809686, 989159646, 1359830474], [3100147259, 3235691256, -2337956167], [18343754, 77244511, 434966358], [2323512370, 2605909356, -1307927025], [2605909356, 2700122696, -1307927025], [2821869347, 3100147259, -466328886], [793913826, 826809686, 3223466997], [2070957389, 2227672101, -1465481009]]
[[2700214958, 3064010529, 43176235], [1484584575, 1509244859, -44511779], [927520818, 1119489869, -492461750], [158

In [ ]:
# Refinement
# [(0, 56), (56, 93), (93, 97), (97, inf)]
# [(0, 1), (1, 70), (70, inf)]

# ->

# [(0, 1), (1, 56), (56, 70), (70, 93), (93, 97), (97, inf)]

In [104]:
def refine_intervals(intervals1, intervals2):
    # Combine and sort the intervals
    combined = sorted(intervals1 + intervals2, key=lambda x: x[0])

    # Initialize the refined list
    refined = []
    end = 0

    for i in range(len(combined)):
        start, stop = combined[i]
        # Update the start to be the maximum of the current start and the last end
        start = max(start, end)

        # If there's a next interval, end at the minimum of the current stop and the next start
        if i + 1 < len(combined):
            next_start = combined[i + 1][0]
            stop = min(stop, next_start)

        # Avoid adding empty intervals
        if start != stop:
            refined.append((start, stop))
            end = stop

    # Handle the interval to infinity
    if refined[-1][1] != float('inf'):
        refined.append((refined[-1][1], float('inf')))

    return refined

# Test with the provided example
intervals1 = [(0, 56), (56, 93), (93, 97), (97, float('inf'))]
intervals2 = [(0, 1), (1, 70), (70, float('inf'))]

refined_intervals = refine_intervals(intervals1, intervals2)
refined_intervals

[(0, 1), (1, 56), (56, 70), (70, 93), (93, 97), (97, inf)]

In [105]:
def pairwise_intersection(ranges):
  """
  Checks if any pair of ranges in the list has an intersection.

  Args:
    ranges: A list of integer ranges represented as tuples (start, end).

  Returns:
    True if any pair of ranges has an intersection, False otherwise.
  """
  for i in range(len(ranges)):
    for j in range(i + 1, len(ranges)):
      range1, range2 = ranges[i], ranges[j]
      if range1[1] > range2[0] and range2[1] > range1[0]:
        return True
  return False

# Example usage
ranges = [(0, 10), (5, 15), (10, 20)]
intersecting = pairwise_intersection(ranges)
print(intersecting)  # Output: True

ranges = [(0, 10), (10, 20), (20, 30)]
intersecting = pairwise_intersection(ranges)
print(intersecting)  # Output: False

True
False
